In [ ]:
!pip install langchain langchain-community transformers sentence-transformers tiktoken chromadb --quiet
!pip install auto-gptq optimum accelerate faiss-cpu --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.0/607.0 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.1 MB/s eta 0:00:0

In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

#Importing Knowledge Base
from langchain.document_loaders import GitbookLoader
import nest_asyncio
import asyncio

from transformers import AutoTokenizer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoModelForCausalLM,AutoTokenizer, pipeline
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub
import optimum
import auto_gptq
import accelerate

In [ ]:
nest_asyncio.apply()
loader = GitbookLoader("https://docs.gitbook.com", load_all_paths=True)
documents = loader.load()

/usr/lib/python3.10/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)
Fetching pages: 100%|##########| 112/112 [00:27<00:00,  4.00it/s]


In [ ]:
print(f"fetched {len(documents)} documents.")

fetched 112 documents.


In [ ]:
model_name_or_path = "TheBloke/Llama-2-7b-Chat-GPTQ"
model_basename = "gptq_model-4bit-128g"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [ ]:
token_counts = [len(tokenizer(page.page_content, return_tensors='pt').input_ids[0])
for page in documents]

In [ ]:
print(f"""Min: {min(token_counts)}
Avg: {int(sum(token_counts) / len(token_counts))}
Max: {max(token_counts)}""")

Min: 11
Avg: 215
Max: 1970


In [ ]:
def tokenizer_len(text):
  return len(tokenizer(text, return_tensors='pt').input_ids[0])

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 10,   #chunk_size +/- allowance to arrive at a separator
    length_function=tokenizer_len,
    separators=['\n\n', '\n', ' ', '']
)

chunks = text_splitter.split_documents(documents)

In [ ]:
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

<ipython-input-15-86f3ffb55a21>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
vectorstore = Chroma.from_documents(documents= chunks,
                                    embedding=embeddings_model)

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map='auto',
                                             trust_remote_code=False,
                                             revision="main")

config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4713: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Llama-2-7b-Chat-GPTQ were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
query = input('The chatbot will assist you with your queries. ')

query_embedding = embeddings_model.embed_query(query)

retrieved_docs = vectorstore.similarity_search_by_vector(query_embedding)

for doc in retrieved_docs:
    print(doc)

The chatbot will assist you with your queries. How do I share my projects on Gitbook?
page_content='Share your content
Discover how you can collaborate with your team when editing your GitBook content.
Last updated 
5 months ago' metadata={'source': 'https://docs.gitbook.com/collaboration/share', 'title': 'Share your content'}
page_content='Share a space
Collaborate with your team on a GitBook space.
Last updated 
1 month ago' metadata={'source': 'https://docs.gitbook.com/collaboration/share/share-a-space', 'title': 'Share a space'}
page_content='Share a collection
Collaborate with your team on a group of spaces within a GitBook collection.
Last updated 
1 month ago' metadata={'source': 'https://docs.gitbook.com/collaboration/share/share-a-collection', 'title': 'Share a collection'}
page_content='Private publishing with share links
Share links give you greater control over who can view your published GitBook documentation.
Last updated 
3 months ago' metadata={'source': 'https://docs.g

In [ ]:
query = "How do I share my projects on Gitbook?"
retrieved_docs = retriever.get_relevant_documents(query)

context = " ".join([doc.page_content for doc in retrieved_docs])
input_text = f"Context: {context}\n\nQuery: {query}"

input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

outputs = model.generate(input_ids, max_length=250)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Context: Share your content
Discover how you can collaborate with your team when editing your GitBook content.
Last updated 
5 months ago Share a space
Collaborate with your team on a GitBook space.
Last updated 
1 month ago Share a collection
Collaborate with your team on a group of spaces within a GitBook collection.
Last updated 
1 month ago

Query: How do I share my projects on Gitbook?

Sharing your GitBook projects with your team is easy. Here's how:

1. Create a space: First, create a space for your team to collaborate on. To do this, go to your GitBook dashboard and click on the "Create Space" button.

2. Invite team members: Once you have created a space, invite your team members to join by sending them an invitation link. You can find this link in the "Members" section of your space settings.

3. Share a project: Once your team members are invited, you can share a project with them by clicking on the "Share" button in the top-right corner of the project. Select
